<a href="https://colab.research.google.com/github/TimyBen/BFS_A-/blob/main/MidTerm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [213]:
from queue import deque
from copy import deepcopy
import heapq

class Node:
    def __init__(self, state, action=None, parent=None, g=0, h=0, f=0):
        self.state = state
        self.action = action
        self.parent = parent
        self.g = g
        self.h = h
        self.f = f
        self.id = str(self.state)

    def get_id(self):
        return self.id

    def get_successors(self):
        successors = []
        empty_row, empty_col = self.get_empty_position()

        # Move the empty tile left
        if empty_col > 0:
            new_state = deepcopy(self.state)
            new_state[empty_row][empty_col], new_state[empty_row][empty_col - 1] = new_state[empty_row][empty_col - 1], new_state[empty_row][empty_col]
            successors.append(Node(new_state, 'LEFT', self))

        # Move the empty tile right
        if empty_col < 2:
            new_state = deepcopy(self.state)
            new_state[empty_row][empty_col], new_state[empty_row][empty_col + 1] = new_state[empty_row][empty_col + 1], new_state[empty_row][empty_col]
            successors.append(Node(new_state, 'RIGHT', self))

        # Move the empty tile up
        if empty_row > 0:
            new_state = deepcopy(self.state)
            new_state[empty_row][empty_col], new_state[empty_row - 1][empty_col] = new_state[empty_row - 1][empty_col], new_state[empty_row][empty_col]
            successors.append(Node(new_state, 'UP', self))

        # Move the empty tile down
        if empty_row < 2:
            new_state = deepcopy(self.state)
            new_state[empty_row][empty_col], new_state[empty_row + 1][empty_col] = new_state[empty_row + 1][empty_col], new_state[empty_row][empty_col]
            successors.append(Node(new_state, 'DOWN', self))

        return successors

    def get_empty_position(self):
        for i in range(len(self.state)):
            for j in range(len(self.state[i])):
                if self.state[i][j] == 0:
                    return i, j
        return None

    def __lt__(self, other):
        return self.f < other.f

    def __le__(self, other):
        return self.f <= other.f

    def __gt__(self, other):
        return self.f > other.f

    def __ge__(self, other):
        return self.f >= other.f

    def __eq__(self, other):
        return self.f == other.f

    def __ne__(self, other):
        return self.f != other.f

    def get_node_str(self):
        return str(self.state)

    def get_action(self):
        return self.action

    def draw(self, dot):
        dot.node(self.get_id(), self.get_node_str())
        if self.parent is not None:
            dot.edge(self.parent.get_id(), self.get_id(), self.get_action())



In [257]:
def is_goal_state(state):
  goal_state = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
  return state == goal_state

def bfs(initial_state):
    start_node = Node(initial_state)
    visited = set()
    queue = deque([start_node])

    while queue:
        current_node = queue.popleft()
        current_state = current_node.state
        visited.add(str(current_state))

        if is_goal_state(current_state):
            return current_node

        for successor in current_node.get_successors():
            successor_state = successor.state
            if str(successor_state) not in visited:
                queue.append(successor)
                visited.add(str(successor_state))

    return None


def astar(initial_state):
    start_node = Node(initial_state)
    visited = set()
    heap = [(0, start_node)]

    while heap:
        _, current_node = heapq.heappop(heap)
        current_state = current_node.state
        current_state_str = str(current_state)  # Convert state to string for hashability
        visited.add(current_state_str)

        if is_goal_state(current_state):
            return current_node

        for successor in current_node.get_successors():
            successor_state = successor.state
            successor_state_str = str(successor_state)  # Convert state to string for hashability
            if successor_state_str not in visited:
                g = current_node.g + 1
                h = heuristic(successor_state)
                f = g + h
                heapq.heappush(heap, (f, successor))
                successor.g = g
                successor.h = h
                successor.f = f

    return None


def heuristic(state):
    # Manhattan distance heuristic
    distance = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0:
                goal_row, goal_col = divmod(state[i][j] - 1, 3)
                distance += abs(i - goal_row) + abs(j - goal_col)
    return distance


In [258]:
def print_solution(node):
    if node is None:
        print("No solution found.")
        return
    actions = []
    while node.parent:
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    print("Solution found with actions:", actions)

# Example usage
initial_state = [[1, 2, 7], [4, 6, 8], [0, 5, 3]]  # Initial state can be arbitrary
bfs_solution = bfs(initial_state)
print("BFS solution:")
print_solution(bfs_solution)

astar_solution = astar(initial_state)
print("\nA* solution:")
print_solution(astar_solution)

BFS solution:
Solution found with actions: ['UP', 'RIGHT', 'DOWN', 'RIGHT', 'UP', 'LEFT', 'DOWN', 'LEFT', 'UP', 'RIGHT', 'DOWN', 'RIGHT', 'UP', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'DOWN', 'LEFT', 'UP', 'UP', 'LEFT']

A* solution:
Solution found with actions: ['UP', 'RIGHT', 'DOWN', 'RIGHT', 'UP', 'LEFT', 'DOWN', 'LEFT', 'UP', 'RIGHT', 'DOWN', 'RIGHT', 'UP', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'DOWN', 'LEFT', 'UP', 'UP', 'LEFT']


In [274]:
from graphviz import Digraph

def visualize_search(initial_state, algorithm):
    dot = Digraph(comment=f'{algorithm} Search')
    dot.format = 'png'

    start_node = Node(initial_state)
    dot.node(start_node.get_id(), start_node.get_node_str())

    if algorithm == 'BFS':
        end_node = bfs(initial_state)
    elif algorithm == 'A*':
        end_node = astar(initial_state)

    path = [end_node]
    while path[-1].parent is not None:
        path.append(path[-1].parent)

    for node in path:
        dot.node(node.get_id(), node.get_node_str())
        if node.parent is not None:
            dot.edge(node.parent.get_id(), node.get_id(), label=node.get_action())

    dot.render(f'{algorithm}_search', cleanup=True)
    return dot, path


# Save steps in markdown format
def save_steps_as_markdown(path, steps):
    with open(path, 'w') as f:
        f.write("# Solution Steps\n\n")
        for i, step in enumerate(steps[::-1], 1):
            state = step.get_node_str()
            action = step.get_action()
            f.write(f"**Step {i}:**\n\n")
            f.write(f"  **0 Move {action}**\n\n")
            f.write(f"**Current State:** {state}")
            f.write(f"\n\n")


In [ ]:
# initial_state = [[2, 7, 1], [4, 6, 8], [0, 5, 3]]  # Initial state can be arbitrary

# bfs_dot, bfs_path = visualize_search(initial_state, 'BFS')
# astar_dot, astar_path = visualize_search(initial_state, 'A*')

# bfs_solution = bfs(initial_state)
# print("BFS solution:")
# bfs_moves = print_solution(bfs_solution)

# astar_solution = astar(initial_state)
# print("\nA* solution:")
# astar_moves = print_solution(astar_solution)
# print()

# save_steps_as_markdown('BFS_steps.md', bfs_path)
# save_steps_as_markdown('A*_steps.md', astar_path)

# print("Visualization saved as BFS_search.png")
# print("Visualization saved as A*_search.png")
# print("Steps saved as BFS_steps.md")
# print("Steps saved as A*_steps.md")

In [276]:
import random
from graphviz import Digraph

# Function to generate a random initial state for the 8-puzzle problem
def generate_random_initial_state():
    numbers = list(range(9))
    random.shuffle(numbers)
    return [numbers[i:i+3] for i in range(0, 9, 3)]

# Function to export the search process as PNG and markdown files
def export_search_process(algorithm, initial_state, solution_node):
    if solution_node is not None:
        dot, path = visualize_search(initial_state, algorithm)
        dot.render(f'{algorithm}_search_{initial_state}', cleanup=True)  # Export the search process as PNG
        save_steps_as_markdown(f'{algorithm}_steps_{initial_state}.md', path)  # Export the search process steps as markdown
        print(f"{algorithm} search process exported successfully for initial state: {initial_state}.")
    else:
        print(f"No solution found for {algorithm} search for initial state: {initial_state}.")

for i in range(2):
  random_state = generate_random_initial_state()
  print(f"\nRandom State {i+1}:")
  for row in random_state:
    print(row)
  bfs_solution = bfs(random_state)
  astar_solution = astar(random_state)

  # Export the search process for BFS and A* algorithms
  export_search_process('BFS', random_state, bfs_solution)
  export_search_process('A*', random_state, astar_solution)


Random State 1:
[7, 2, 6]
[4, 8, 3]
[1, 5, 0]
No solution found for BFS search for initial state: [[7, 2, 6], [4, 8, 3], [1, 5, 0]].
No solution found for A* search for initial state: [[7, 2, 6], [4, 8, 3], [1, 5, 0]].

Random State 2:
[6, 3, 0]
[5, 4, 2]
[1, 8, 7]
BFS search process exported successfully for initial state: [[6, 3, 0], [5, 4, 2], [1, 8, 7]].
A* search process exported successfully for initial state: [[6, 3, 0], [5, 4, 2], [1, 8, 7]].
